## ECE 188 Unfinished Musical Project

This part of the project basically runs on the google collab. This is just the example code that I use, and also to show the plots of the results. The link for the google collab is in the technical implementation portion of the README.md in this github.

The goal of this part is to generate the continuation as well as accompaniment of musical pieces

Before, we begin anything we have to perform the environmental setup.

# Setting Up

In [0]:
!gsutil -q -m cp -r gs://magentadata/models/music_transformer/* /content/
!gsutil -q -m cp gs://magentadata/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2 /content/
!apt-get update -qq && apt-get install -qq libfluidsynth1 build-essential libasound2-dev libjack-dev
!pip install -qU magenta pyfluidsynth

import ctypes.util
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return ctypes.util.find_library(lib)
ctypes.util.find_library = proxy_find_library

import numpy as np
import os
import tensorflow as tf

from google.colab import files

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.data_generators import text_encoder
from tensor2tensor.utils import decoding
from tensor2tensor.utils import trainer_lib

import magenta.music as mm
from magenta.models.score2perf import score2perf

SF2_PATH = '/content/Yamaha-C5-Salamander-JNv5.1.sf2'
SAMPLE_RATE = 16000

def upload_midi():
  data = list(files.upload().values())
  if len(data) > 1:
    print('Multiple files uploaded; using only one.')
  return mm.midi_to_note_sequence(data[0])

def decode(ids, encoder):
  ids = list(ids)
  if text_encoder.EOS_ID in ids:
    ids = ids[:ids.index(text_encoder.EOS_ID)]
  return encoder.decode(ids)

### Set up the Melody-conditioned Transformer model
Skip this part if the accompaniment for the music is not missing.

In [0]:
model_name = 'transformer'
hparams_set = 'transformer_tpu'
ckpt_path = '/content/checkpoints/melody_conditioned_model_16.ckpt'

class MelodyToPianoPerformanceProblem(score2perf.AbsoluteMelody2PerfProblem):
  @property
  def add_eos_symbol(self):
    return True

problem = MelodyToPianoPerformanceProblem()
melody_conditioned_encoders = problem.get_feature_encoders()
hparams = trainer_lib.create_hparams(hparams_set=hparams_set)
trainer_lib.add_problem_hparams(hparams, problem)
hparams.num_hidden_layers = 16
hparams.sampling_method = 'random'
decode_hparams = decoding.decode_hparams()
decode_hparams.alpha = 0.0
decode_hparams.beam_size = 1
run_config = trainer_lib.create_run_config(hparams)
estimator = trainer_lib.create_estimator(
    model_name, hparams, run_config,
    decode_hparams=decode_hparams)

inputs = []
decode_length = 0

def input_generator():
  global inputs
  while True:
    yield {
        'inputs': np.array([[inputs]], dtype=np.int32),
        'targets': np.zeros([1, 0], dtype=np.int32),
        'decode_length': np.array(decode_length, dtype=np.int32)
    }

input_fn = decoding.make_input_fn_from_generator(input_generator())
melody_conditioned_samples = estimator.predict(
    input_fn, checkpoint_path=ckpt_path)

_ = next(melody_conditioned_samples)

In [0]:
event_padding = 2 * [mm.MELODY_NO_EVENT]
melody = 'Upload'  

if melody == 'Upload':

  melody_ns = upload_midi()
  melody_instrument = mm.infer_melody_for_sequence(melody_ns)
  notes = [note for note in melody_ns.notes
           if note.instrument == melody_instrument]
  del melody_ns.notes[:]
  melody_ns.notes.extend(
      sorted(notes, key=lambda note: note.start_time))
  for i in range(len(melody_ns.notes) - 1):
    melody_ns.notes[i].end_time = melody_ns.notes[i + 1].start_time
  inputs = melody_conditioned_encoders['inputs'].encode_note_sequence(
      melody_ns)
else:

  events = [event + 12 if event != mm.MELODY_NO_EVENT else event
            for e in melodies[melody]
            for event in [e] + event_padding]
  inputs = melody_conditioned_encoders['inputs'].encode(
      ' '.join(str(e) for e in events))
  melody_ns = mm.Melody(events).to_sequence(qpm=150)
  
mm.plot_sequence(melody_ns)


### The plots of the generated sequence can be examine below.
The files can also be listen through the 'input' folder as well as the 'sample generated output' folder in each of the titled songs folders.

This plot here is the piece from TAPS (Butterfield's Lullaby)

<img src="Images/Butterfield_1.png" alt="plot" style="width: 450px;"/>

### Generate the accompaniment

In [0]:
decode_length = 4096
sample_ids = next(melody_conditioned_samples)['outputs']

midi_filename = decode(
    sample_ids,
    encoder=melody_conditioned_encoders['targets'])
accompaniment_ns = mm.midi_file_to_note_sequence(midi_filename)

mm.plot_sequence(accompaniment_ns)

<img src="Images/Butterfield_2.png" alt="plot" style="width: 500px;"/>

In [0]:
mm.sequence_proto_to_midi_file(
    accompaniment_ns, '/tmp/TAPS_accompaniment.mid')
files.download('/tmp/TAPS_accompaniment.mid')

# Load the piano performance language model

In [0]:
model_name = 'transformer'
hparams_set = 'transformer_tpu'
ckpt_path = '/content/checkpoints/unconditional_model_16.ckpt'

class PianoPerformanceLanguageModelProblem(score2perf.Score2PerfProblem):
  @property
  def add_eos_symbol(self):
    return True

problem = PianoPerformanceLanguageModelProblem()
unconditional_encoders = problem.get_feature_encoders()

hparams = trainer_lib.create_hparams(hparams_set=hparams_set)
trainer_lib.add_problem_hparams(hparams, problem)
hparams.num_hidden_layers = 16
hparams.sampling_method = 'random'

decode_hparams = decoding.decode_hparams()
decode_hparams.alpha = 0.0
decode_hparams.beam_size = 1

run_config = trainer_lib.create_run_config(hparams)
estimator = trainer_lib.create_estimator(
    model_name, hparams, run_config,
    decode_hparams=decode_hparams)

def input_generator():
  global targets
  global decode_length
  while True:
    yield {
        'targets': np.array([targets], dtype=np.int32),
        'decode_length': np.array(decode_length, dtype=np.int32)
    }

targets = []
decode_length = 0

input_fn = decoding.make_input_fn_from_generator(input_generator())
unconditional_samples = estimator.predict(
    input_fn, checkpoint_path=ckpt_path)

_ = next(unconditional_samples)

In [0]:
primer = 'Upload'  #Upload the songs.

if primer == 'Upload':
  primer_ns = upload_midi()

primer_ns = mm.apply_sustain_control_changes(primer_ns)

max_primer_seconds = 6  
if primer_ns.total_time > max_primer_seconds:
  print('Primer is longer than %d seconds, truncating.' % max_primer_seconds)
  primer_ns = mm.extract_subsequence(
      primer_ns, 0, max_primer_seconds)

if any(note.is_drum for note in primer_ns.notes):
  print('Primer contains drums; they will be removed.')
  notes = [note for note in primer_ns.notes if not note.is_drum]
  del primer_ns.notes[:]
  primer_ns.notes.extend(notes)

for note in primer_ns.notes:
  note.instrument = 1
  note.program = 0

## Generate Improvisation and plot


In [0]:
targets = unconditional_encoders['targets'].encode_note_sequence(
    primer_ns)

targets = targets[:-1]

decode_length = max(0, 4096 - len(targets))
if len(targets) >= 4096:
  print('No generation...')

sample_ids = next(unconditional_samples)['outputs']

midi_filename = decode(
    sample_ids,
    encoder=unconditional_encoders['targets'])
ns = mm.midi_file_to_note_sequence(midi_filename)

continuation_ns = mm.concatenate_sequences([primer_ns, ns])

mm.plot_sequence(continuation_ns)



#### The files can also be listen through the 'input' folder as well as the 'sample generated output' folder in each of the titled songs folders.


Generated improvisation for Butterfield's Lullaby piece.

<img src="Images/Butterfield_4.png" alt="plot" style="width: 500px;"/>

<img src="Images/Butterfield_5.png" alt="plot" style="width: 470px;"/>

Generated improvisation for Mozart's Requiem.

<img src="Images/Mozart_1.png" alt="plot" style="width: 500px;"/>

Generated improvisation for Turandot.

<img src="Images/Turandot_1.png" alt="plot" style="width: 470px;"/>

<img src="Images/Turandot_2.png" alt="plot" style="width: 490px;"/>